# Letzte Tests vor Training

Ziel dieses Notebooks ist es, die Modellierungsbasis für das Projekt Rookie Invest zu validieren.

Wir prüfen:
1. Datenstruktur und Abdeckung
2. Eindeutigkeit der Schlüssel
3. Erstellung der Zielvariable (F1 Einstieg nach Saison t)
4. Leakage Risiken (Zeitlogik, Identifier, Serien)
5. Duplikate in F2/F3 und saubere Deduplizierung
6. Erstellung der finalen Trainingsdatei in `data/model_input/`

Wichtig: Dieses Notebook verändert keine bestehenden Dateien in `data/*/processed`. Es erzeugt nur eine neue Datei in `data/model_input/`.


In [14]:
from pathlib import Path
import pandas as pd
from IPython.display import display


# Projekt-Root (eine Ebene über notebooks/)
PROJECT_ROOT = Path.cwd().parent

PATH_ALL = PROJECT_ROOT / "data/all_series/processed/all_series_master_features_core.csv"
OUT_DIR = PROJECT_ROOT / "data/model_input"
OUT_PATH = OUT_DIR / "f2_f3_features_with_f1_label.csv"

print("Using project root:", PROJECT_ROOT)
print("CSV exists:", PATH_ALL.exists())


Using project root: /Users/sheyla/Desktop/rookie_invest_ML
CSV exists: True


## Zielvariable

Wir modellieren die Wahrscheinlichkeit für einen F1 Einstieg basierend auf Performance bis Saison t.

Definition:
- Für jede Fahrer Saison Zeile in F2 oder F3 gilt `f1_entry = 1`, wenn derselbe Fahrer in einem späteren Jahr in der F1 auftaucht.
- `f1_entry = 0`, wenn kein späterer F1 Einstieg gefunden wird.
- Fälle, in denen der erste F1 Einstieg im gleichen Jahr wie die F2/F3 Saison liegt, werden aus dem Training entfernt, weil das nicht "Zukunft" ist.


In [5]:
#Daten laden und Grundcheck

df = pd.read_csv(PATH_ALL)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head(5))


Shape: (3713, 26)
Columns: ['series', 'year', 'driver_name', 'driver_code', 'team_name', 'n_races', 'total_points', 'avg_points', 'avg_finish', 'best_finish', 'worst_finish', 'wins', 'win_rate', 'podiums', 'podium_rate', 'points_finishes', 'points_rate', 'top10_finishes', 'top10_rate', 'total_laps', 'avg_kph', 'finish_std', 'points_std', 'dnf_count', 'dnf_rate', 'avg_best_lap_s']


,series,year,driver_name,driver_code,team_name,n_races,total_points,avg_points,avg_finish,best_finish,...,points_rate,top10_finishes,top10_rate,total_laps,avg_kph,finish_std,points_std,dnf_count,dnf_rate,avg_best_lap_s
0,F1,1950,Alberto Ascari,\N,Ferrari,4,11.0,2.2,8.6,2,...,0.75,3,0.75,238.0,NaN,7.765307,2.48998,4.0,1.0,NaN
1,F1,1950,Alfredo Pián,\N,Maserati,1,0.0,0.0,21.0,21,...,0.00,0,0.00,0.0,NaN,NaN,NaN,1.0,1.0,NaN
2,F1,1950,Bayliss Levrett,\N,Adams,1,0.0,0.0,27.0,27,...,0.00,0,0.00,108.0,NaN,NaN,NaN,1.0,1.0,NaN
3,F1,1950,Bill Cantrell,\N,Adams,1,0.0,0.0,27.0,27,...,0.00,0,0.00,108.0,NaN,NaN,NaN,1.0,1.0,NaN
4,F1,1950,Bill Holland,\N,Deidt,1,6.0,6.0,2.0,2,...,1.00,1,1.00,137.0,NaN,NaN,NaN,1.0,1.0,NaN


In [6]:
#Serien und Jahr Abdeckung
print(df["series"].value_counts())
print("Year range:", df["year"].min(), "-", df["year"].max())


series
F1    3211
F3     255
F2     247
Name: count, dtype: int64
Year range: 1950 - 2025


## Schlüssel und Duplikate

Wir erwarten für F2 und F3 eine Zeile pro Fahrer und Saison.
Für F1 Altjahre kann `driver_code` fehlen. Das ist für das Training nicht relevant, solange F2/F3 sauber sind.


In [7]:
#Duplikate global und in F2/F3
KEY = ["series", "year", "driver_code"]

print("Global duplicates by (series, year, driver_code):", df.duplicated(KEY).sum())

non_f1 = df[df["series"].isin(["F2","F3"])].copy()

print("F2+F3 shape:", non_f1.shape)
print("driver_code equals \\N in F2+F3:", (non_f1["driver_code"] == r"\N").sum())
print("Duplicates F2/F3 by (series, year, driver_code):", non_f1.duplicated(KEY).sum())
print("Duplicates F2/F3 by (series, year, driver_name):", non_f1.duplicated(["series","year","driver_name"]).sum())


Global duplicates by (series, year, driver_code): 2558
F2+F3 shape: (502, 26)
driver_code equals \N in F2+F3: 0
Duplicates F2/F3 by (series, year, driver_code): 10
Duplicates F2/F3 by (series, year, driver_name): 5


## F1 Einstieg Jahr

Wir bestimmen pro Fahrer den ersten F1 Jahrgang anhand der F1 Zeilen.
Diese Information wird ausschliesslich für das Label verwendet und nicht als Feature in das Modell gegeben.


In [8]:
#First F1 year erzeugen
f1 = df[df["series"] == "F1"].copy()
first_f1_year = f1.groupby("driver_code")["year"].min()

print("Drivers with F1 entry:", first_f1_year.shape[0])
display(first_f1_year.head(10))


Drivers with F1 entry: 98


driver_code
AIT    2020
ALB    2005
ALG    2009
ALO    2001
BAD    1993
BAR    1993
BEA    2024
BIA    1959
BOT    2013
BOU    2008
Name: year, dtype: int64

## Label Erstellung und Zeitlogik

Wir erstellen:
- `first_f1_year`
- `f1_entry` als bool

Sanity Checks:
- Verteilung der Labels
- Anzahl same year Konflikte (`first_f1_year == year`)


In [9]:
#Label bauen
tmp = non_f1.copy()
tmp["first_f1_year"] = tmp["driver_code"].map(first_f1_year)
tmp["f1_entry"] = tmp["first_f1_year"].notna() & (tmp["first_f1_year"] > tmp["year"])

print(tmp["f1_entry"].value_counts())
print("Positive share:", tmp["f1_entry"].mean())

same_year_conflicts = ((tmp["first_f1_year"] == tmp["year"]) & tmp["first_f1_year"].notna()).sum()
print("Same-year conflicts:", same_year_conflicts)


f1_entry
False    470
True      32
Name: count, dtype: int64
Positive share: 0.06374501992031872
Same-year conflicts: 4


## Duplikate in F2/F3 anzeigen

Wir schauen uns die Zeilen an, die im selben Jahr, in derselben Serie, mit demselben Fahrer mehrfach vorkommen.
Typische Ursachen sind Teamwechsel oder Datenfehler.


In [10]:
#Duplikat Zeilen ausgeben
dup = tmp[tmp.duplicated(["series","year","driver_code"], keep=False)].copy()
print("Duplicated rows:", dup.shape[0])

cols_show = ["series","year","driver_name","driver_code","team_name","n_races","total_points","wins","podiums","dnf_count"]
display(
    dup.sort_values(["series","year","driver_code","n_races","total_points"], ascending=[True,True,True,False,False])[cols_show]
)


Duplicated rows: 19


,series,year,driver_name,driver_code,team_name,n_races,total_points,wins,podiums,dnf_count
3469,F3,2019,F Scherer,SCH,Sauber Junior Team by Charouz,8,NaN,1,3,NaN
3466,F3,2019,D Schumacher,SCH,Campos Racing,1,NaN,0,0,NaN
3504,F3,2020,D Schumacher,SCH,Charouz Racing System,6,NaN,1,1,NaN
3503,F3,2020,D Schumacher,SCH,Carlin Buzz Racing,3,NaN,0,1,NaN
3563,F3,2021,Z Chovanec,CHO,Charouz Racing System,3,NaN,0,0,NaN
3556,F3,2021,P Chovet,CHO,Campos Racing,1,NaN,0,1,NaN
3557,F3,2021,P Chovet,CHO,Jenzer Motorsport,1,NaN,0,0,NaN
3602,F3,2022,Z Maloney,MAL,Trident,9,NaN,0,2,NaN
3575,F3,2022,F Malvestiti,MAL,Jenzer Motorsport,8,NaN,0,0,NaN
3585,F3,2022,J Martí,MAR,Campos Racing,9,NaN,1,1,NaN


## Deduplizierung Regel

Entscheid:
Wir behalten pro (series, year, driver_code) genau eine Zeile.
Regel:
- bevorzugt die Zeile mit den meisten Rennen (`n_races`)
- bei Gleichstand die mit mehr Punkten (`total_points`)

Begründung:
Mehr Rennen bedeutet stabilere Saisonstatistik und reduziert Verzerrungen im Training.


In [11]:
#Deduplizieren, Label neu, same year raus
clean = non_f1.copy()

clean = (
    clean.sort_values(["series","year","driver_code","n_races","total_points"], ascending=[True,True,True,False,False])
         .drop_duplicates(["series","year","driver_code"], keep="first")
         .copy()
)

clean["first_f1_year"] = clean["driver_code"].map(first_f1_year)
clean["f1_entry"] = clean["first_f1_year"].notna() & (clean["first_f1_year"] > clean["year"])

# remove same-year conflicts
clean = clean[~((clean["first_f1_year"] == clean["year"]) & clean["first_f1_year"].notna())].copy()

print("After dedupe shape:", clean.shape)
print(clean["f1_entry"].value_counts())
print("Positive share:", clean["f1_entry"].mean())


After dedupe shape: (488, 28)
f1_entry
False    456
True      32
Name: count, dtype: int64
Positive share: 0.06557377049180328


## Leakage Guardrails

Für das Training entfernen wir alle Identifier Spalten:
- driver_name
- driver_code
- team_name
- series

Wir behalten:
- year für Splits und Analyse, nicht als starkes Feature
- rein numerische Performance Features


In [12]:
#Feature Set definieren
DROP_COLS = ["driver_name", "driver_code", "team_name", "series"]

feature_cols = [c for c in clean.columns if c not in DROP_COLS + ["f1_entry", "first_f1_year"]]

print("Number of features:", len(feature_cols))
print(feature_cols)


Number of features: 22
['year', 'n_races', 'total_points', 'avg_points', 'avg_finish', 'best_finish', 'worst_finish', 'wins', 'win_rate', 'podiums', 'podium_rate', 'points_finishes', 'points_rate', 'top10_finishes', 'top10_rate', 'total_laps', 'avg_kph', 'finish_std', 'points_std', 'dnf_count', 'dnf_rate', 'avg_best_lap_s']


## Export nach model_input

Wir speichern eine neue Datei in `data/model_input/`.
Bestehende Dateien werden nicht verändert.


In [13]:
#Speichern
OUT_DIR.mkdir(parents=True, exist_ok=True)

clean.to_csv(OUT_PATH, index=False)

print("Saved:", OUT_PATH)
print("Saved shape:", clean.shape)


Saved: /Users/sheyla/Desktop/rookie_invest_ML/data/model_input/f2_f3_features_with_f1_label.csv
Saved shape: (488, 28)


## Ergebnis

Wenn alle Checks ohne Auffälligkeiten laufen, ist der Datensatz bereit für das Training.
Nächster Schritt:
- zeitbasierter Train Test Split
- Modelltraining mit class weights und geeigneten Metriken (ROC AUC, PR AUC)


In [16]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parent  # weil Notebook in notebooks/
DATA_PATH = PROJECT_ROOT / "data/model_input/f2_f3_features_with_f1_label.csv"

print("Looking for:", DATA_PATH)
print("Exists:", DATA_PATH.exists())

df = pd.read_csv(DATA_PATH)

print(df.shape)
print(df["f1_entry"].value_counts())
print(df.isna().sum().sort_values(ascending=False).head(5))


Looking for: /Users/sheyla/Desktop/rookie_invest_ML/data/model_input/f2_f3_features_with_f1_label.csv
Exists: True
(488, 28)
f1_entry
False    456
True      32
Name: count, dtype: int64
first_f1_year      372
points_std         265
points_finishes    244
dnf_rate           244
dnf_count          244
dtype: int64
